# Libraries & Settings

In [16]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import qgrid
#import beakerx as bx
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, store_excel
from pa_lib.data import (calc_col_partitioned, clean_up_categoricals, flatten, 
                         replace_col, cond_col, desc_col, unfactorize)
from pa_lib.util import obj_size, cap_words
from pa_lib.log  import time_log, info
from pa_lib.types import dtFactor
from pa_lib.vis import dive

# display long columns completely
pd.set_option('display.max_colwidth', 200)

pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [17]:
bd_raw = load_bin('bd_data.feather').rename(mapper=lambda name: cap_words(name, sep='_'), axis='columns')
bd = bd_raw.loc[(bd_raw.Netto >= 0)].pipe(clean_up_categoricals)

2019-07-15 13:35:40 [INFO] Reading from file C:\Users\kpf\data\bd_data.feather
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
2019-07-15 13:35:40 [INFO] Finished loading binary file in 0.15s (0.56s CPU)


In [18]:
desc_col(bd)

,DTYPE,NULLS,UNIQUE
Endkunde_Nr,category,0/1526093,41740
Endkunde,category,0/1526093,39122
Ek_Abc,category,5389/1520704,8
Ek_Boni,category,2408/1523685,3
Ek_Plz,category,475/1525618,3468
Ek_Ort,category,474/1525619,3504
Ek_Land,category,474/1525619,60
Ek_Hb_Apg_Kurzz,category,67234/1458859,95
Ek_Aktiv,category,0/1526093,2
Agentur,category,770186/755907,4606


# Prepare Endkunden Information

In [53]:
def collect(s, sep=', '):
    return sep.join(map(str, s[s.notna()].unique()))

ek_info = (bd.sort_values(['Endkunde_Nr', 'Kampagne_Erfassungsdatum'])
           .groupby('Endkunde_Nr')
           .agg({'Endkunde': 'last',
                 'Ek_Aktiv': 'last',
                 'Ek_Land': 'last',
                 'Ek_Plz': 'last',
                 'Ek_Ort': 'last',
                 'Agentur': 'last',
                 'Endkunde_Branchengruppe': 'last',
                 'Endkunde_Branchengruppe_Id': 'last',
                 'Auftrag_Branchengruppe_Id': [collect, 'nunique'],
                 'Kamp_Erfass_Jahr': ['min', 'max']
                 }))

ek_info

Endkunde Ek_Aktiv  \
                                                     last     last   
Endkunde_Nr                                                          
100034                               Maurer + Salzmann AG        1   
100039                            Bank BSU Genossenschaft        1   
100043                  Werbeagentur Knaus & Knaus BSW AG        0   
100045                               Stoz Werbeagentur AG        1   
100061                               Mundwiler Juwelen AG        1   
100064                                 Senn Communication        1   
100066                     walder, werber werbeagentur ag        1   
100068                    Naef Marketing Kommunikation AG        1   
100077                                           Romer AG        1   
100083                    Goodyear Dunlop Tires Suisse SA        1   
100092                                        Navyboot AG        1   
100095                                           Zomag AG        1   
100097                                          Biomed AG        1   
100098                         Maison Astor Coiffure GmbH        1   
100104                                Huser Automobile AG        1   
100109                         Mietervereinigung Illuster        1   
100112                                       Setaprint AG        1   
100119                                         Initiative        1   
100121                                 MZ Möbelzentrum AG        1   
100128                                     Christinger AG        1   
100135                                        Qualipet AG        1   
100137                                Gamper Siebdruck AG        0   
100143                    Mietervereinigung EKZ Volkiland        1   
100145                                 Huonder + Elmer AG        1   
100148                              Limmatdruck Zeiler AG        1   
100156                                  Body Gym Uster AG        1   
100166                        Mietervereinigung Neuwiesen        1   
100179                    Orlando Duò Graphic Design GmbH        1   
100190                                   Eskimo Textil AG        1   
100194                                          NetVision        0   
100196                                         Gabriel AG        1   
100199                                  Garage Neuburg AG        1   
100202                                   Möbel Ferrari AG        1   
100204                            Heineken Switzerland AG        1   
100205                              Verein Greifenseelauf        1   
100211                           Restaurant Waldmannsburg        1   
100216                      Korean Motor Company, Kontich        1   
100227                Team HP Schneider AG für Gestaltung        1   
100229                            the Generation Media AG        1   
100231                    Swiss Science Center Technorama        1   
100236           Goal AG für Werbung und Public Relations        1   
100239                                 Industriegarage AG        1   
100246                                 Garage Quaranta AG        1   
100247                             smartcut consulting AG        1   
100248                               Bolli Textilwaren AG        1   
100258                                   Universal McCann        1   
100265                   kinetic worldwide Switzerland ag        1   
100273                   Genossenschaft Migros Ostschweiz        0   
100277                                     Schwaninger AG        1   
100314                      Publicis Media Switzerland AG        1   
100320                                  Alpstein-Drogerie        1   
100326                     AMAG Automobil- und Motoren AG        1   
100335       AMW - Werbewerkstatt der fenaco-LANDI Gruppe        1   
100338                          Atelier für Kommunikation        1   
100348                              Fotomuseum Winterthur  

### Distribution of Auftragsart vs. Vertrag

In [77]:
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr',aggfunc='size', fill_value=0))
display('Netto-Umsatz')
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr', values='Netto', aggfunc='sum', fill_value=0))

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,90,2074,1698,1815,2337,2786,1520,33,16,19,23,14
Eigenwerbung APG,Nein,0,0,294,69,124,177,41,721,1256,1380,1168,423,138
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,2786,4069,1704
Goodwill,Nein,0,10,103,104,123,180,163,134,192,134,150,158,61
Karitativ,Nein,0,200,438,695,403,704,524,726,897,1301,1354,1541,746
Kommerziell,Nein,0,14150,71746,78234,84716,78514,77696,91767,108510,126063,125936,126373,74202
Logistik für Dritte,Nein,0,0,96,525,1434,1416,1292,1530,1900,1866,2045,1772,810
Politisch,Nein,0,1041,3880,4385,8983,5950,5655,6170,10853,9185,6759,7973,6969
Promotion,Nein,0,191,3895,2870,3123,3301,5370,2590,3584,4673,5462,7906,988


'Netto-Umsatz'

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,30444,849600,791206,739514,1026394,1000669,439344,110412,92214,117954,150966,72198
Eigenwerbung APG,Nein,0,0,3897655,0,4670,1048532,0,360,133390,126540,1222454,72427,0
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,0,0,0
Goodwill,Nein,0,0,6543,7939,4953,18668,2994,4762,4191,1732,17772,16741,5925
Karitativ,Nein,0,724644,1960077,1628917,1866012,2668615,2253943,2318474,2578634,3459117,2775404,3962032,2632140
Kommerziell,Nein,0,33806952,148051046,162393013,168628594,175031787,158519973,182560236,193519765,204222602,197001081,191949492,145654263
Logistik für Dritte,Nein,0,0,0,0,4600,184578,31589,132394,479850,420927,418245,522661,239121
Politisch,Nein,0,2040430,6938949,6463858,18562173,11082670,9945225,10458567,16977360,11414298,6379246,8489070,10458705
Promotion,Nein,0,1083041,5375644,4376195,3393104,3505266,5451654,3185928,3787542,4991362,4881049,9304271,1960374
